In [5]:
import csv
from random import shuffle
import os

total_patients = 10
num_folds = 10
patients_test = 2

out_sub_folder = "folds_test"

random_assignment = True

prefix = "s"
suffix = "_norm_windows_20"


subjects_path = "../envelops/min-max/windows_20/tr-True_sliding_20_c-False/"
log = subjects_path + out_sub_folder + "/folds_info.txt"
if not os.path.exists(subjects_path + out_sub_folder):
    os.mkdir(subjects_path + out_sub_folder)

with open(log, 'w') as f_log: 

    for n in range(1,num_folds+1):
        train_file = subjects_path + out_sub_folder + "/TrainFold_" + str(n) + ".csv"
        test_file_learned = subjects_path + out_sub_folder + "/TestFold_L_" + str(n) + ".csv"
        test_file_unlearned = subjects_path + out_sub_folder + "/TestFold_U_" + str(n) + ".csv"

        if random_assignment:
            l = [i for i in range(1,total_patients+1)]
            shuffle(l)
            l_test = l[:patients_test]
            l_train = l[patients_test:]
        else:
            l_test = []
            for i in range((n-1)*patients_test +1,(n-1)*patients_test + patients_test+1):
                if (i<=total_patients):
                    l_test.append(i)
            l_train = []
            for i in range(1,total_patients+1):
                if not i in l_test:
                    l_train.append(i)
                    
        print("Fold: " + str(n))    
        print("Train: " + str(l_train))
        print("Test: " + str(l_test))
        
        
        with open(train_file, mode='w') as t_out:
            csv_writer = csv.writer(t_out, delimiter=',')
            with open(test_file_learned, mode='w') as t_out_test:
                csv_writer_test = csv.writer(t_out_test, delimiter=',')
                f_log.write("Subjects in Train Fold " + str(n) + ": " + str(l[5:]) + "\n") 
                for i in l_train:
                    with open(subjects_path + prefix + str(i) + suffix +'.csv', mode='r') as s_in:
                        csv_reader = csv.reader(s_in, delimiter=',')
                        row_count = sum(1 for row in csv_reader)
                    train_count = int(row_count*0.9)
                    print("Rows: " + str(row_count))
                    print("Train rows:" + str(train_count))
                    count = 0
                    with open(subjects_path + prefix + str(i) + suffix +'.csv', mode='r') as s_in:
                        csv_reader = csv.reader(s_in, delimiter=',')
                        for line in csv_reader:
                            count += 1
                            if (count<train_count):
                                csv_writer.writerow(line)
                            else:
                                csv_writer_test.writerow(line)
        with open(test_file_unlearned, mode='w') as t_out:
            csv_writer = csv.writer(t_out, delimiter=',')
            f_log.write("Subjects in Test Fold " + str(n) + ": " + str(l[:5])+ "\n\n") 
            for i in l_test:
                with open(subjects_path + prefix + str(i) + suffix +'.csv', mode='r') as s_in:
                    csv_reader = csv.reader(s_in, delimiter=',')
                    for line in csv_reader:
                        csv_writer.writerow(line)
        print("Fold " + str(n) + " node!")
        
        


Fold: 1
Train: [10, 4, 3, 6, 5, 7, 9, 8]
Test: [1, 2]
Rows: 28278
Train rows:25450
Rows: 27642
Train rows:24877
Rows: 26658
Train rows:23992
Rows: 38010
Train rows:34209
Rows: 26012
Train rows:23410
Rows: 25580
Train rows:23022
Rows: 29700
Train rows:26730
Rows: 27264
Train rows:24537
Fold 1 node!
Fold: 2
Train: [3, 10, 8, 2, 9, 7, 1, 4]
Test: [6, 5]
Rows: 26658
Train rows:23992
Rows: 28278
Train rows:25450
Rows: 27264
Train rows:24537
Rows: 31344
Train rows:28209
Rows: 29700
Train rows:26730
Rows: 25580
Train rows:23022
Rows: 31150
Train rows:28035
Rows: 27642
Train rows:24877
Fold 2 node!
Fold: 3
Train: [9, 10, 7, 5, 6, 2, 3, 4]
Test: [8, 1]
Rows: 29700
Train rows:26730
Rows: 28278
Train rows:25450
Rows: 25580
Train rows:23022
Rows: 26012
Train rows:23410
Rows: 38010
Train rows:34209
Rows: 31344
Train rows:28209
Rows: 26658
Train rows:23992
Rows: 27642
Train rows:24877
Fold 3 node!
Fold: 4
Train: [2, 9, 5, 10, 6, 3, 8, 7]
Test: [4, 1]
Rows: 31344
Train rows:28209
Rows: 29700
Train ro